# Ultimate Fandoms

In [1]:
# Imports for the whole Analysis

# imports
from urllib.request import urlopen
import json
import re
from urllib.parse import quote
import pandas as pd
from pandas.io.json import json_normalize
import collections
import itertools
import networkx as nx
import matplotlib.pyplot as plt
from fa2 import ForceAtlas2
import numpy as np
import datetime
from lxml import html
import glob
import ast # string to dirc
import math # check if float is NaN

In [2]:
# Get wikis/dataset from csv
fullDataSet = pd.read_csv("data/sortedWikiData.csv", sep='\t')

# Drop old index column named "Unnamed: 0"
fullDataSet = fullDataSet.drop(['Unnamed: 0'], axis=1)

In [3]:
# Methode that remove inactive wikis and duplicates
def findActiveWikis(dataSet):
    activeDataSet = pd.DataFrame()
    
    # Remove duplicates
    dataSet = dataSet.drop_duplicates()
    
    # Find stats in dataset
    for wiki in dataSet.T.items():
        wiki = wiki[1]
        stats = wiki['stats']
        
        # Only add wiki to list when at in contains at least 1 active user
        sDirc = ast.literal_eval(stats)
        if int(sDirc['activeUsers'])>1:
            activeDataSet = activeDataSet.append(wiki) 
    
    # Create new index
    activeDataSet = activeDataSet.reset_index(drop=True)
    return activeDataSet

In [4]:
### Filter Wiki Data ###
activeDataSet = findActiveWikis(fullDataSet)

print("%s Wikis before filter, and %s Wikis efter filter.\n%s has been removed" % 
      (len(fullDataSet),len(activeDataSet),len(fullDataSet)-len(activeDataSet)))

# Data memory usage
print(sum(activeDataSet.memory_usage(index=False, deep=True))/1000)

activeDataSet.head()

35039 Wikis before filter, and 5585 Wikis efter filter.
29454 has been removed
7978.467


,creation_date,desc,domain,flags,founding_user_id,headline,hub,id,image,lang,language,name,original_dimensions,stats,title,topUsers,topic,url,wam_score,wordmark
0,2006-01-10 15:06:36,NaN,broadcasting.wikia.com,[],12423.0,NaN,TV,942.0,NaN,en,en,Radio-TV Broadcast History,NaN,"{'edits': 24380, 'articles': 4362, 'pages': 56...",Radio-TV Broadcast History,"[4106832, 27024879, 36887098, 37504326, 268032...",Education,http://broadcasting.wikia.com,2.4801,NaN
1,2006-05-27 12:27:18,NaN,zh.tv.wikia.com,[],23838.0,NaN,TV,1592.0,NaN,zh-tw,zh-tw,中文電視大典,NaN,"{'edits': 18338, 'articles': 977, 'pages': 403...",中文電視大典,"[36879666, 36808293, 36843235, 4784321, 581119...",TV,http://zh.tv.wikia.com,4.7445,NaN
2,2007-11-12 08:47:09,Greek is an ABC comedy series following studen...,greektv.wikia.com,[],245831.0,Welcome to the Greek Wiki!,TV,4477.0,https://images.wikia.nocookie.net/__cb20161102...,en,en,Greek TV Series Wiki,"{'width': 480, 'height': 320}","{'edits': 1687, 'articles': 82, 'pages': 558, ...",Greek TV Series Wiki,"[27262712, 35817224, 30088906, 37296348, 33484...",Humor,http://greektv.wikia.com,1.0088,https://vignette4.wikia.nocookie.net/greektv/i...
3,2009-05-29 21:43:42,NaN,smtvlive.wikia.com,[],1393647.0,NaN,TV,26398.0,NaN,en,en,SM:tv Live Wiki,NaN,"{'edits': 794, 'articles': 41, 'pages': 406, '...",SM:tv Live Wiki,"[36603313, 37317816, 32946464, 37031283, 37105...",TV,http://smtvlive.wikia.com,0.0000,https://images.wikia.com/smtvlive/images/8/89/...
4,2009-07-18 15:01:24,Here at the OZ TV Wikia community we dedicated...,oztv.wikia.com,[],1469765.0,Learn more about the Oz TV Wikia,TV,37717.0,https://images.wikia.nocookie.net/__cb20161102...,en,en,Oz TV Wiki,"{'width': 500, 'height': 320}","{'edits': 7750, 'articles': 370, 'pages': 2124...",Oz TV Wiki,"[32455677, 35064581, 25687838, 1469765, 293452...",Entertainment,http://oztv.wikia.com,3.7153,https://images.wikia.com/oztv/images/8/89/Wiki...


In [46]:
###### Save article ID data in hub-search-articles-hub-date #####
date = datetime.date.today().strftime("%d-%m-%Y")
base_query = "http://www.wikia.com/api/v1/Articles/TopByHub?hub="
ns_all = ['110','1']

for hub in set(activeDataSet["hub"]):
    if str(hub) == 'nan':
            continue
    for ns in ns_all:

        query = base_query+hub+"&namespaces="+ns
        response = urlopen(query)
        wikisource = response.read()
        data = json.loads(wikisource)
        print(hub, ns, '\n',
              len(data['items']),
              len(data['items'][0]['articles']), 
              '\n', query)
        #[print(articles['articles']) for articles in data['items']]
    
        # Save data at data/wikis/top-wikis-date.json
        with open("data/articles/hub-search-articles-"+hub+"-"+date+'-'+ns+".json", 'w') as outfile:
            json.dump(data, outfile)
    

Music 110 
 4 19 
 http://www.wikia.com/api/v1/Articles/TopByHub?hub=Music&namespaces=110
Music 1 
 9 20 
 http://www.wikia.com/api/v1/Articles/TopByHub?hub=Music&namespaces=1
Movies 110 
 8 20 
 http://www.wikia.com/api/v1/Articles/TopByHub?hub=Movies&namespaces=110
Movies 1 
 10 20 
 http://www.wikia.com/api/v1/Articles/TopByHub?hub=Movies&namespaces=1
Books 110 
 8 20 
 http://www.wikia.com/api/v1/Articles/TopByHub?hub=Books&namespaces=110
Books 1 
 10 20 
 http://www.wikia.com/api/v1/Articles/TopByHub?hub=Books&namespaces=1
Games 110 
 8 20 
 http://www.wikia.com/api/v1/Articles/TopByHub?hub=Games&namespaces=110
Games 1 
 10 20 
 http://www.wikia.com/api/v1/Articles/TopByHub?hub=Games&namespaces=1
Lifestyle 110 
 3 2 
 http://www.wikia.com/api/v1/Articles/TopByHub?hub=Lifestyle&namespaces=110
Lifestyle 1 
 8 20 
 http://www.wikia.com/api/v1/Articles/TopByHub?hub=Lifestyle&namespaces=1
Other 110 
 8 20 
 http://www.wikia.com/api/v1/Articles/TopByHub?hub=Other&namespaces=110
Other 1 

In [47]:
def searchForArticles(domain, article_id_list):
    article_id_string = '%2C'.join(article_id_list)
    query = domain+"/api/v1/Articles/Details?ids="+article_id_string+"&abstract=500&width=200&height=200"
    response = urlopen(query)
    wikisource = response.read()
    return json.loads(wikisource)

In [48]:
def searchForArticlesSimple(domain, article_id_list):
    error = 0
    data = []
    for article_id in article_id_list:
        try:
            query = domain+"/api/v1/Articles/AsSimpleJson?id="+str(article_id)
            response = urlopen(query)
            wikisource = response.read()
            data.append(json.loads(wikisource))
        except:
            print(domain, article_id, 'error')
            error +=1
    print(len(data), error)
    return data

In [49]:
###### Save inital data in top-wikis-date #####
def find_articles(data):
    data = ast.literal_eval(str(data))
    data = data['items']
    for wiki in data:
        wiki_data = wiki['wiki']
        domain = wiki_data['domain']
        name = wiki_data['name']
        
        wiki_id = str(wiki_data['id'])
        articles = []
        article_data = []
            
        for article in wiki['articles']:
            articles.append(str(article['id']))
            
        print(name)
        article_data.append(searchForArticlesSimple(domain, articles))
            #wiki_article_ids = name+' ['+wiki_id+']: '
            
            #sections = article_data['sections']
            #for article in sections:
            #    print(article['title'])
            #    for content in article['content']:
            #        if 'text' in content:
            #            print(content['text'])
            #        else:
            #            print(content)
            #print()
    return article_data

In [50]:
for dataPath in glob.glob("data/articles/*.json"):
    print('_______________', dataPath)
    with open(dataPath) as f:
        json_file = json.loads(f.read())
    file_name = dataPath.replace('articles/','text/text-')
    #file_name = file_name.replace('.json','.txt')
    with open(file_name, 'w') as outfile:
        json.dump(find_articles(json_file), outfile)

_______________ data/articles/hub-search-articles-Books-09-12-2018-1.json
Old School RuneScape Wiki
http://oldschoolrunescape.wikia.com 0 error
19 1
Fallout Wiki
http://fallout.wikia.com 0 error
19 1
Elder Scrolls
http://elderscrolls.wikia.com 0 error
19 1
WARFRAME Wiki
http://warframe.fandom.com 1373207 error
http://warframe.fandom.com 1372715 error
http://warframe.fandom.com 1372550 error
http://warframe.fandom.com 1372468 error
16 4
Harry Potter Wiki
http://harrypotter.wikia.com 0 error
19 1
Yu-Gi-Oh!
20 0
RuneScape Wiki
http://runescape.fandom.com 0 error
19 1
Wookieepedia
http://starwars.wikia.com 0 error
19 1
WikiDex
20 0
D&D 5th Edition Wikia
5 0
_______________ data/articles/hub-search-articles-Books-09-12-2018-110.json
Old School RuneScape Wiki
http://oldschoolrunescape.wikia.com 0 error
19 1
Fallout Wiki
20 0
Elder Scrolls
20 0
WARFRAME Wiki
8 0
Harry Potter Wiki
20 0
Yu-Gi-Oh!
20 0
RuneScape Wiki
20 0
WikiDex
20 0
_______________ data/articles/hub-search-articles-Comics-09-1

In [38]:
for dataPath in glob.glob("data/text/*.json"):
    print('_______________', dataPath)
    with open(dataPath) as f:
        json_file = json.loads(f.read())
    for section in json_file:
        for s in section:
            for b in s['sections']:
                print('# # # ', b['title'])
                #print(b['level'])
                for c in b['content']:
                    if c['type'] == 'paragraph':
                        print('', c['text'])
                    elif c['type'] == 'list':
                        [print('-', d['text']) for d in c['elements']]
                        
                    else:
                        print('ERROR?!?!', c)
                        
            print()

_______________ data/text/text-hub-search-articles-Books-09-12-2018-1.json
# # #  False Life/@comment-37236120-20181019145809/@comment-35014500-20181019160710
 It would be 1d4+9. 
 Each additional level gives you 5 more HP on top of the exising value. So by 9th level you would be gaining 1d4+44 HP (if you wanted to waste your only slot on this anyways).
 This spell is slightly odd in that regard since usually you just gain more dice. However the tradeoff is that you aren't gambling on the extra health, it will always be a minimum of the flat value added.
 Edit: corrected 9th level HP value since I calculated it wrong before.

# # #  D&D 5th Edition Wikia
 I've been away for a while and have thus been unable to address this. If you look at the disclaimers on the homepage of the wiki, it notes that not all information is available, due to lack of licensing for 5th edition. As of now (July 2015), Wizards of the Coast has yet to release a license for 5th edition, meaning anything that is N

 ATT: Densort
 Muchísimas gracias por la lista. Justo lo que necesitaba.
 ATT: Victor
 por fin encuentro la guía de como ver pokémon en orden muchas gracias
 ATT: BRYAM
# # #  Actualicenlo
 deberian actualizarlo a xy, ya salio la nueva generacion, asi que por lo mientras estaria bien saber cuales siguen en cuanto a kalos. Buen post por cierto

# # #  Como hago la evolucion de amistad o amor?? Budew
 xFavor, quiero saber como hacer evolucionar a mi Budew en un Roselia y al final en Roserade, segun los datos de Budew dice que le gusta lo seco, no se que bayas hay que darle ni cuantas y tambien quiero saber en donde encuentro una roca luz para que al final evolucione en Roserade, un no consigo mi segunda medalla porque aun no llego al pueblo en donde la concigo, lo que digo es que hasta ahorita estoy comenzando con el juego, les agradeceria una pronta respuesta... SERGIO4444 (discusión) 14:57 23 nov 2008 (UTC)SERGIOSERGIO4444 (discusión) 14:57 23 nov 2008 (UTC)

# # #  ¿que son los IV,s y

- Use "c" for circa instead of circa, about ,abt, etc.
- Use "aft" for after, later than, etc.
- Use "bef" for before, earlier than, etc.
- No spaces between the c, aft, bef and the dates (Use John Smith (c1850-bef1920) instead of John Smith (c 1850-bef 1920)
- No spaces between the dates and the dash. Use (1900-1985) instead of (1900 - 1985)
- Support
- Support -AMK152 00:20, 16 December 2007 (UTC)
- Support - Will 05:14, 16 December 2007 (UTC)
- I support the idea about normalization of notation. On the other hand, I have a great deal of ambivalence about what goes into this portion of the name. For example, I am still unconvinced that death date should be mandatory. It seems to me it just creates more reason for people to be renaming articles. ~ Phlox 17:59, 18 December 2007 (UTC)
- Same qualified support as Phlox's. See Genealogy talk:Page names. Robin Patterson 22:43, 1 January 2008 (UTC)
- Oppose
- Neutral
# # #  Redirects
 Problem: We have alot of people articles out there that 

- Additional Information: Leader
# # #  MARIN
- Real name: Suga Marin (菅真鈴)
- Birthdate: November 21, 1997 (1997-11-21) (age 21)
- Official Color: Orange
- Favorite food: Curry rice
- Hobbies: Sleeping, writing lyrics
- Special Skill: playing the piano
- Additional Information: Main vocalist in PEACEFUL
# # #  MANAKA
- Real name: Inaba Manaka (稲場愛香)
- Nickname: Ma~nyan (ま～にゃん)
- Birthdate: December 27, 1997 (1997-12-27) (age 20)
- Birthplace: Hokkaido, Japan
- Bloodtype: B
- Official Color: Yellow (KAKAMASASHI color: Black)
- Hobbies: music appreciation, reading, painting/drawing, having wild ideas
- Special Skills: dance, making people laugh, English level 5.
- Favorite Food: Fruits
- Additional Information: Sub-leader, trained at EXPG school in Sapporo, back-danced for EXILE, joined Hello Pro Kenshuusei - debuted as a member of Country Girls.
# # #  SAYO
- Real name: Nodake Sayo (野獄沙世)
- Nickname: Sayo-chin (さよちん)
- Birthdate: December 26, 1997 (1997-12-26) (age 20)
- Official Color:

 --Majin Buu (discusión) 04:40 30 mar 2011 (UTC)
 Si quieres un consejo, no captures a Heracross, hay muchos más mejor que él. Pero si lo quieres para conseguir todos los Pokémon de la Pokédex, deberías seguir intentandolo en Pueblo Azalea que es más fácil, aunque la proabilidad de que salga es la misma que si golpeas un árbol de esas rutas. Lo de "cabezear rocas" será usar golpe roca para partir rocas y que salgan Pokémon. Un saludo del experto en Acero, Roca y Tierra. --Mpl160 (discusión) 13:04 30 mar 2011 (UTC)

# # #  Creación de un artículo de como llenar la pokedex
 Yo siempre he jugado a los videojuegos y siempre me he preguntado como llenar la Pokedex, lo que no quiero es como obtenerlo, lo que me gustaría que implementen es un artículo en donde describan de forma detallada como VER al pokémon para así llenar la Pokedex. Por ejemplo:
 -Un Venusaur, lo tiene el joven de la ruta 123, pelea contra él y ya lo tendrás en la Pokedex.
 Para nada es imposible, yo en el Blanco lo estoy 

In [ ]:
#query = 'http://fallout.wikia.com/d/f?sort=trending'
#response = urlopen(query)
#wikisource = response.read()
#regstring = "http://fallout.wikia.com/d/p/3343172654596187821"
#usernames = re.findall(regstring,str(wikisource))
#wikisource

In [ ]:
#query = 'http://fallout.wikia.com/d/p/3343172654596187821'
#response = urlopen(query)
#wikisource = response.read()
#regstring = "<div class=\"post-content\">(.*?)(<\/div>|<div)"
#usernames = re.findall(regstring,str(wikisource))
#[print(a[0], end='\n\n') for a in usernames]